In [1]:
import os
import re
import glob
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
path = "/Users/tracysanchezpacheco/Desktop/*.json"
## Extracting all subject IDs from the data folder
subIDs = []
for sub in glob.glob(path):
    if sub[35].isdigit():
        subIDs.append(int(sub[35:39]))
    else:
        pass
subIDs = np.unique(subIDs)
print(subIDs)

[8521]


In [3]:
# create empty dataframe
df = pd.DataFrame()

# read every file name in folder
for filename in glob.glob(path):
    with open(filename, 'r') as file:
        # make json files parsable
        data = "[" + file.read()
        data = data[:len(data)-1] + "]"

        # read data per file
        subjectdf = pd.read_json(data)

        # insert participant id in every line
        subjectdf.insert(0, "SubjectID", [int(filename[35:39])] * subjectdf.shape[0], True)
        df = df.append(subjectdf, ignore_index=True)
df.head(8)

,SubjectID,ParticipantID,TaskOrder,Language,InitialOrientation,PointingTaskRandomised,PointingTaskStartingLocations,PointingTaskTargets_1,PointingTaskTargets_2,PointingTaskTargets_3,...,PointingTaskTargets_19,PointingTaskTargets_20,PointingTaskTargets_21,PointingTaskTargets_22,PointingTaskTargets_23,PointingTaskTargets_24,PointingTaskTargets_25,PointingTaskTargets_26,PointingTaskTargets_27,PointingTaskTargets_28
0,8521,8521,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...","[22, 20, 14, 25, 7, 2, 21, 1, 9, 24, 4, 6, 5, ...","[38, 14, 30, 0, 53, 50]","[43, 23, 42, 6, 34, 23]","[8, 20, 17, 38, 25, 16]",...,"[24, 55, 50, 3, 36, 46]","[13, 52, 16, 45, 8, 5]","[27, 44, 37, 9, 33, 4]","[28, 7, 0, 12, 12, 55]","[34, 46, 24, 16, 41, 38]","[33, 31, 7, 33, 14, 36]","[18, 42, 30, 22, 12, 23]","[7, 37, 15, 34, 30, 20]","[29, 31, 35, 38, 12, 42]","[50, 25, 45, 22, 10, 33]"


In [4]:
StartingPoints = pd.DataFrame(df.explode('PointingTaskStartingLocations', ignore_index=True))
StartingPoints.head(29)
StartingPoints.shape

(28, 35)

In [5]:
StartingPoints = pd.DataFrame(df.explode('PointingTaskStartingLocations'))

In [6]:
StartingPoints['StartPointID']= np.tile(np.arange(1,29,1), len(StartingPoints.ParticipantID.unique()))
StartingPoints.head()

,SubjectID,ParticipantID,TaskOrder,Language,InitialOrientation,PointingTaskRandomised,PointingTaskStartingLocations,PointingTaskTargets_1,PointingTaskTargets_2,PointingTaskTargets_3,...,PointingTaskTargets_20,PointingTaskTargets_21,PointingTaskTargets_22,PointingTaskTargets_23,PointingTaskTargets_24,PointingTaskTargets_25,PointingTaskTargets_26,PointingTaskTargets_27,PointingTaskTargets_28,StartPointID
0,8521,8521,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",22,"[38, 14, 30, 0, 53, 50]","[43, 23, 42, 6, 34, 23]","[8, 20, 17, 38, 25, 16]",...,"[13, 52, 16, 45, 8, 5]","[27, 44, 37, 9, 33, 4]","[28, 7, 0, 12, 12, 55]","[34, 46, 24, 16, 41, 38]","[33, 31, 7, 33, 14, 36]","[18, 42, 30, 22, 12, 23]","[7, 37, 15, 34, 30, 20]","[29, 31, 35, 38, 12, 42]","[50, 25, 45, 22, 10, 33]",1
0,8521,8521,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",20,"[38, 14, 30, 0, 53, 50]","[43, 23, 42, 6, 34, 23]","[8, 20, 17, 38, 25, 16]",...,"[13, 52, 16, 45, 8, 5]","[27, 44, 37, 9, 33, 4]","[28, 7, 0, 12, 12, 55]","[34, 46, 24, 16, 41, 38]","[33, 31, 7, 33, 14, 36]","[18, 42, 30, 22, 12, 23]","[7, 37, 15, 34, 30, 20]","[29, 31, 35, 38, 12, 42]","[50, 25, 45, 22, 10, 33]",2
0,8521,8521,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",14,"[38, 14, 30, 0, 53, 50]","[43, 23, 42, 6, 34, 23]","[8, 20, 17, 38, 25, 16]",...,"[13, 52, 16, 45, 8, 5]","[27, 44, 37, 9, 33, 4]","[28, 7, 0, 12, 12, 55]","[34, 46, 24, 16, 41, 38]","[33, 31, 7, 33, 14, 36]","[18, 42, 30, 22, 12, 23]","[7, 37, 15, 34, 30, 20]","[29, 31, 35, 38, 12, 42]","[50, 25, 45, 22, 10, 33]",3
0,8521,8521,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",25,"[38, 14, 30, 0, 53, 50]","[43, 23, 42, 6, 34, 23]","[8, 20, 17, 38, 25, 16]",...,"[13, 52, 16, 45, 8, 5]","[27, 44, 37, 9, 33, 4]","[28, 7, 0, 12, 12, 55]","[34, 46, 24, 16, 41, 38]","[33, 31, 7, 33, 14, 36]","[18, 42, 30, 22, 12, 23]","[7, 37, 15, 34, 30, 20]","[29, 31, 35, 38, 12, 42]","[50, 25, 45, 22, 10, 33]",4
0,8521,8521,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",7,"[38, 14, 30, 0, 53, 50]","[43, 23, 42, 6, 34, 23]","[8, 20, 17, 38, 25, 16]",...,"[13, 52, 16, 45, 8, 5]","[27, 44, 37, 9, 33, 4]","[28, 7, 0, 12, 12, 55]","[34, 46, 24, 16, 41, 38]","[33, 31, 7, 33, 14, 36]","[18, 42, 30, 22, 12, 23]","[7, 37, 15, 34, 30, 20]","[29, 31, 35, 38, 12, 42]","[50, 25, 45, 22, 10, 33]",5


In [7]:
ExploteID = []
for item in range(1,29):
    step = 'StartPointID == '  + str(item)
    ExploteID.append(step)

In [8]:
# Create a list of the column names of variables to explote
ColumnList = list(StartingPoints.iloc[:,7:35].columns)

In [9]:
# Create dictionary 
ExploteDic = dict(zip(ExploteID, ColumnList))
ExploteDic

{'StartPointID == 1': 'PointingTaskTargets_1',
 'StartPointID == 2': 'PointingTaskTargets_2',
 'StartPointID == 3': 'PointingTaskTargets_3',
 'StartPointID == 4': 'PointingTaskTargets_4',
 'StartPointID == 5': 'PointingTaskTargets_5',
 'StartPointID == 6': 'PointingTaskTargets_6',
 'StartPointID == 7': 'PointingTaskTargets_7',
 'StartPointID == 8': 'PointingTaskTargets_8',
 'StartPointID == 9': 'PointingTaskTargets_9',
 'StartPointID == 10': 'PointingTaskTargets_10',
 'StartPointID == 11': 'PointingTaskTargets_11',
 'StartPointID == 12': 'PointingTaskTargets_12',
 'StartPointID == 13': 'PointingTaskTargets_13',
 'StartPointID == 14': 'PointingTaskTargets_14',
 'StartPointID == 15': 'PointingTaskTargets_15',
 'StartPointID == 16': 'PointingTaskTargets_16',
 'StartPointID == 17': 'PointingTaskTargets_17',
 'StartPointID == 18': 'PointingTaskTargets_18',
 'StartPointID == 19': 'PointingTaskTargets_19',
 'StartPointID == 20': 'PointingTaskTargets_20',
 'StartPointID == 21': 'PointingTaskTa

In [10]:
def explote_multiple(dataframe, dic):
    
    """ Creates a list of dataframes that include one exploted variable based row conditionals derivated from
    keys, values contain of a Dictionary
    
Parameters
-------------
     
     :dataframe: your data frame,
     :dic: Dictionary contains a list of key values that will inform a query of the df
     and values that contain the names of columns to explote. 
returns 
----------

    List of DataFrames
     """

    tempdf = [] #Empty list to store each exploted data set
    for conditional, variable_to_explote in dic.items():
        data_q = dataframe.query(conditional).explode(variable_to_explote)
        ques = pd.DataFrame(data_q)
        tempdf.append(ques)
    data_frames_list = pd.concat(tempdf, axis=0)
    return data_frames_list
#%%
ExploteAplication = explote_multiple(StartingPoints, ExploteDic)
ExploteAplication.shape

(168, 36)

In [11]:
ExploteAplication.set_index(['ParticipantID', 'StartPointID'], inplace= True)
#%%
# Probe of concept how to just retain integers AKA the exploted data
b = pd.DataFrame( ExploteAplication['PointingTaskTargets_1'][ExploteAplication['PointingTaskTargets_1'].map(type)==int])
b.tail()

PointingTaskTargets_1
ParticipantID StartPointID                      
8521          1                               14
              1                               30
              1                                0
              1                               53
              1                               50

In [12]:
def integrate_explote(dataframe, list_variables):
    empty = []
    for l in  list_variables:
        step = dataframe[l][dataframe[l].map(type)==int]
        empty.append(step)
    finals = pd.DataFrame(pd.concat(empty, axis=0), columns= ['Target'])
    dataframe['Trials'] = finals['Target']
    return dataframe

In [13]:
Integrated = integrate_explote(ExploteAplication, ColumnList)
Integrated.info()
print(Integrated.tail())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 168 entries, (8521, 1) to (8521, 28)
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   SubjectID                      168 non-null    int64 
 1   TaskOrder                      168 non-null    object
 2   Language                       168 non-null    object
 3   InitialOrientation             168 non-null    object
 4   PointingTaskRandomised         168 non-null    object
 5   PointingTaskStartingLocations  168 non-null    object
 6   PointingTaskTargets_1          168 non-null    object
 7   PointingTaskTargets_2          168 non-null    object
 8   PointingTaskTargets_3          168 non-null    object
 9   PointingTaskTargets_4          168 non-null    object
 10  PointingTaskTargets_5          168 non-null    object
 11  PointingTaskTargets_6          168 non-null    object
 12  PointingTaskTargets_7          168 non-null    ob

In [14]:
Integrated.sort_index(level= ['ParticipantID'], inplace=True)
Integrated.head()

SubjectID       TaskOrder Language  \
ParticipantID StartPointID                                       
8521          1                  8521  [PointingTask]  English   
              1                  8521  [PointingTask]  English   
              1                  8521  [PointingTask]  English   
              1                  8521  [PointingTask]  English   
              1                  8521  [PointingTask]  English   

                                                           InitialOrientation  \
ParticipantID StartPointID                                                      
8521          1             {'x': 0.0008186895865940001, 'y': -0.906321287...   
              1             {'x': 0.0008186895865940001, 'y': -0.906321287...   
              1             {'x': 0.0008186895865940001, 'y': -0.906321287...   
              1             {'x': 0.0008186895865940001, 'y': -0.906321287...   
              1             {'x': 0.0008186895865940001, 'y': -0.906321287...   

                                                       PointingTaskRandomised  \
ParticipantID StartPointID                                                      
8521          1             [{'Block': 1, 'LocationNumber': 1, 'StartingPo...   
              1             [{'Block': 1, 'LocationNumber': 1, 'StartingPo...   
              1             [{'Block': 1, 'LocationNumber': 1, 'StartingPo...   
              1             [{'Block': 1, 'LocationNumber': 1, 'StartingPo...   
              1             [{'Block': 1, 'LocationNumber': 1, 'StartingPo...   

                           PointingTaskStartingLocations  \
ParticipantID StartPointID                                 
8521          1                                       22   
              1                                       22   
              1                                       22   
              1                                       22   
              1                                       22   

                           PointingTaskTargets_1    PointingTaskTargets_2  \
ParticipantID StartPointID                                                  
8521          1                               38  [43, 23, 42, 6, 34, 23]   
              1                               14  [43, 23, 42, 6, 34, 23]   
              1                               30  [43, 23, 42, 6, 34, 23]   
              1                                0  [43, 23, 42, 6, 34, 23]   
              1                               53  [43, 23, 42, 6, 34, 23]   

                              PointingTaskTargets_3     PointingTaskTargets_4  \
ParticipantID StartPointID                                                      
8521          1             [8, 20, 17, 38, 25, 16]  [19, 24, 27, 45, 20, 32]   
              1             [8, 20, 17, 38, 25, 16]  [19, 24, 27, 45, 20, 32]   
              1             [8, 20, 17, 38, 25, 16]  [19, 24, 27, 45, 20, 32]   
              1             [8, 20, 17, 38, 25, 16]  [19, 24, 27, 45, 20, 32]   
              1             [8, 20, 17, 38, 25, 16]  [19, 24, 27, 45, 20, 32]   

                            ...  PointingTaskTargets_20  \
ParticipantID StartPointID  ...                           
8521          1             ...  [13, 52, 16, 45, 8, 5]   
              1             ...  [13, 52, 16, 45, 8, 5]   
              1             ...  [13, 52, 16, 45, 8, 5]   
              1             ...  [13, 52, 16, 45, 8, 5]   
              1             ...  [13, 52, 16, 45, 8, 5]   

                            PointingTaskTargets_21  PointingTaskTargets_22  \
ParticipantID StartPointID                                                   
8521          1             [27, 44, 37, 9, 33, 4]  [28, 7, 0, 12, 12, 55]   
              1             [27, 44, 37, 9, 33, 4]  [28, 7, 0, 12, 12, 55]   
              1             [27, 44, 37, 9, 33, 4]  [28, 7, 0, 12, 12, 55]   
              1             [27, 44, 37, 9, 33, 4]  [28, 7, 0, 12, 12, 55]   
              1   

In [16]:
Targets = Integrated.loc[:,['SubjectID','PointingTaskStartingLocations','Trials']]
Targets.head(224)

SubjectID PointingTaskStartingLocations Trials
ParticipantID StartPointID                                                
8521          1                  8521                            22     38
              1                  8521                            22     14
              1                  8521                            22     30
              1                  8521                            22      0
              1                  8521                            22     53
...                               ...                           ...    ...
              28                 8521                            17     25
              28                 8521                            17     45
              28                 8521                            17     22
              28                 8521                            17     10
              28                 8521                            17     33

[168 rows x 3 columns]